## 6.4 Custom Conversation Chain

<div style="text-align: right"> Initial issue : 2025.05.12 </div>
<div style="text-align: right"> last update : 2025.05.12 </div>

Conversation chain을 클래스로 정의

In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, Runnable
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [3]:
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

In [4]:
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

/tmp/ipykernel_74982/2844409624.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")


In [5]:
class MyConversationChain(Runnable):
    
    def __init__(self, llm, prompt, memory, input_key="input"):

        self.prompt = prompt
        self.memory = memory
        self.input_key = input_key

        self.chain = (
            RunnablePassthrough.assign(
                chat_history=RunnableLambda(self.memory.load_memory_variables)
                | itemgetter(memory.memory_key)  # memory_key 와 동일하게 입력합니다.
            )
            | prompt
            | llm
            | StrOutputParser()
        )

    def invoke(self, query, configs=None, **kwargs):
        answer = self.chain.invoke({self.input_key: query})
        self.memory.save_context(inputs={"human": query}, outputs={"ai": answer})
        return answer

테스트

In [6]:
conversation_chain = MyConversationChain(llm, prompt, memory)

In [7]:
conversation_chain.invoke("안녕하세요? 만나서 반갑습니다. 제 이름은 홍길동입니다.")

'안녕하세요, 홍길동님! 만나서 반갑습니다. 어떻게 도와드릴까요?'

In [8]:
conversation_chain.invoke("제 이름이 뭐라고요?")

'당신의 이름은 홍길동입니다. 맞나요?'